In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import optuna
from tqdm import tqdm

# 1) 데이터 불러오기
train = pd.read_csv("../../data/processed/processed_train.csv")
test = pd.read_csv("../../data/processed/processed_test.csv")

# 2) X, y 분리
X = train.drop(columns=['ID', '성공확률'])
y = train['성공확률']

# 3) 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test = test.drop(columns=['ID'])
X_test_scaled = scaler.transform(X_test)

# Optuna 목적 함수
def objective(trial):
    lr = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    epochs = trial.suggest_int('epochs', 50, 150)
    dropout_rates = [trial.suggest_float(f'dropout{i}', 0.1, 0.6) for i in range(1, 5)]
    units = [trial.suggest_int(f'units{i}', 32, 256) for i in range(1, 5)]
    n_layers = trial.suggest_int('n_layers', 3, 4)

    model = Sequential()
    model.add(Input(shape=(X_scaled.shape[1],)))

    for i in range(n_layers):
        model.add(Dense(units[i], activation='relu'))
        model.add(Dropout(dropout_rates[i]))

    model.add(Dense(1))

    model.compile(optimizer=Adam(learning_rate=lr),
                  loss='mean_absolute_error',
                  metrics=['mean_absolute_error'])

    early_stop = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

    model.fit(X_scaled, y, epochs=epochs, batch_size=batch_size,
              verbose=0, callbacks=[early_stop])

    loss = model.evaluate(X_scaled, y, verbose=0)[0]
    return loss

# Optuna 스터디 & tqdm 연동
n_trials = 50
study = optuna.create_study(direction='minimize')

with tqdm(total=n_trials) as pbar:
    def tqdm_callback(study, trial):
        pbar.update(1)
    study.optimize(objective, n_trials=n_trials, callbacks=[tqdm_callback])

print("Best trial:")
trial = study.best_trial
print(f"  Value (MAE): {trial.value}")
print("  Params:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

# 베스트 하이퍼파라미터로 최종 모델 학습 및 예측
best_params = trial.params

model = Sequential()
model.add(Input(shape=(X_scaled.shape[1],)))
for i in range(best_params['n_layers']):
    model.add(Dense(best_params[f'units{i+1}'], activation='relu'))
    model.add(Dropout(best_params[f'dropout{i+1}']))
model.add(Dense(1))

model.compile(optimizer=Adam(learning_rate=best_params['learning_rate']),
              loss='mean_absolute_error',
              metrics=['mean_absolute_error'])

early_stop = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

model.fit(X_scaled, y, epochs=best_params['epochs'], batch_size=best_params['batch_size'],
          verbose=1, callbacks=[early_stop])

# 테스트 데이터 예측
y_test_pred = model.predict(X_test_scaled).flatten()

# 제출 파일 생성
submission = pd.DataFrame({
    'ID': test['ID'],
    '성공확률': y_test_pred
})
submission['성공확률'] = submission['성공확률'].clip(0, 1)  # 필요 시 클리핑

submission.to_csv('../../data/output/submission.csv', index=False)

[I 2025-05-23 19:23:53,067] A new study created in memory with name: no-name-1cf475f6-d460-4474-a62d-3e047ccd3e9c
100%|██████████| 50/50 [15:26<00:00, 18.53s/it]

Best trial:
  Value (MAE): 0.06239191070199013
  Params:
    learning_rate: 0.000996057232413194
    batch_size: 16
    epochs: 142
    dropout1: 0.1207632731556716
    dropout2: 0.13513792259110022
    dropout3: 0.14600939088919987
    dropout4: 0.3614094031186118
    units1: 187
    units2: 216
    units3: 164
    units4: 88
    n_layers: 4
Epoch 1/142


274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 914us/step - loss: 0.3163 - mean_absolute_error: 0.3163
Epoch 2/142
274/274 ━━━━━━━━━━━━━━━━━━━━ 0s 934us/step - loss: 0.2713 - mean_absolute_error: 0.2713
Epoch 3/142
274/274 ━━━━━━━━━━━━━━━━━━━━ 0s 914us/step - loss: 0.2681 - mean_absolute_error: 0.2681
Epoch 4/142
274/274 ━━━━━━━━━━━━━━━━━━━━ 0s 901us/step - loss: 0.2538 - mean_absolute_error: 0.2538
Epoch 5/142
274/274 ━━━━━━━━━━━━━━━━━━━━ 0s 900us/step - loss: 0.2534 - mean_absolute_error: 0.2534
Epoch 6/142
274/274 ━━━━━━━━━━━━━━━━━━━━ 0s 895us/step - loss: 0.2442 - mean_absolute_error: 0.2442
Epoch 7/142
274/274 ━━━━━━━━━━━━━━━━━━━━ 0s 918us/step - loss: 0.2472 - mean_absolute_error: 0.2472
Epoch 8/142
274/274 ━━━━━━━━━━━━━━━━━━━━ 0s 896us/step - loss: 0.2444 - mean_absolute_error: 0.2444
Epoch 9/142
274/274 ━━━━━━━━━━━━━━━━━━━━ 0s 916us/step - loss: 0.2334 - mean_absolute_error: 0.2334
Epoch 10/142
274/274 ━━━━━━━━━━━━━━━━━━━━ 0s 971us/step - loss: 0.2350 - mean_absolute_error: 0.2350
Epoch 11/14

In [10]:
submission

,ID,성공확률
0,TEST_0000,0.441260
1,TEST_0001,0.329026
2,TEST_0002,0.639163
3,TEST_0003,0.311008
4,TEST_0004,0.861093
...,...,...
1750,TEST_1750,0.710341
1751,TEST_1751,0.547599
1752,TEST_1752,0.322759
1753,TEST_1753,0.698343


In [17]:
train['성공확률'].value_counts()

성공확률
0.750    590
0.625    579
0.250    574
0.375    561
0.500    559
0.875    548
0.000    490
1.000    475
Name: count, dtype: int64

# 프로젝트 계획 요약

## 1. 데이터 이해 및 전처리
- 성공확률이 0.1 단위로 구분된 이산형 값임을 확인
- 범주형 변수 원핫 인코딩, 수치형 변수 정규화 적용
- 성공확률을 클래스(분류 레이블)로 변환 (예: 0.1 → 클래스 1, 0.2 → 클래스 2 ...)

## 2. 문제 정의
- 성공확률 예측 → 회귀보다는 분류 문제로 전환하여 접근
- 다중 클래스 분류 문제로 모델 학습

## 3. 모델링
- 신경망 모델 설계 (MLP 기반 다중 은닉층)
- 하이퍼파라미터 튜닝 (Optuna 활용)
  - 은닉층 수, 유닛 수, 드롭아웃 비율, 학습률, 배치사이즈, 에포크 범위 등 폭넓게 설정
- 조기 종료(EarlyStopping) 적용

## 4. 학습 및 검증
- 교차검증 또는 Hold-out 검증으로 과적합 방지 및 일반화 평가
- MAE 외 분류 정확도 등 다양한 평가 지표 활용

## 5. 예측 및 제출
- 테스트 데이터에 대해 클래스 예측 수행
- 예측 클래스 → 성공확률 값으로 매핑하여 결과 생성
- 제출 파일 생성 및 평가

## 6. 향후 개선 및 확장
- 분류+회귀 혼합 모델 고려
- 샘플 가중치, 손실 함수 커스터마이징 등 불균형 및 비대칭 분포 대응 전략 적용
- 다양한 신경망 아키텍처 실험 (CNN, Transformer 등)